In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/headhunter-employer-review-competition/HeadHunter_train.csv
/kaggle/input/headhunter-employer-review-competition/HeadHunter_sample_submit.csv
/kaggle/input/headhunter-employer-review-competition/HeadHunter_test.csv


In [2]:
import re
from tqdm import tqdm
import transformers
%matplotlib inline

from sklearn.model_selection import train_test_split
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
words = stopwords.words("russian")
lemma = nltk.stem.WordNetLemmatizer()


import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup

import random
import time

In [3]:
train_df = pd.read_csv('/kaggle/input/headhunter-employer-review-competition/HeadHunter_train.csv')

In [4]:
train_df.head()

,review_id,city,position,positive,negative,salary_rating,team_rating,managment_rating,career_rating,workplace_rating,rest_recovery_rating,target
0,0,Ижевск,Кладовщик,Недавно устроился в ****** на должность кладов...,Повысить з/п,5,4,3,3,4,4,0
1,1,Москва,Руководитель проекта,"Расположение офиса, своего рода стабильность (...",Упростить процессы - все процессы ради процесс...,4,4,4,1,1,1,8
2,2,Екатеринбург,Консультант,В нашем банке все по закону в плане зарплаты и...,Ничего,5,5,5,5,5,5,0
3,5,Санкт-Петербург,Оператор,Я в ****** около года работаю оператором. Норм...,Нет.,5,5,5,5,5,5,0
4,7,Москва,Инженер КИПиА,В принципе норм.Работать можно,Всегда есть к чему стремиться,4,5,4,2,5,2,8


In [5]:
train_df['Target_vector'] = train_df.target.apply(lambda target: [1  if number in [int(x) for x in target if x.isdigit()] else 0 for number in range(9)])

In [6]:
train_df = train_df[(~train_df.positive.isna())]
train_df = train_df[(~train_df.negative.isna())]

In [7]:
print(train_df.positive.apply(lambda x: len(x)).mean(), train_df.positive.apply(lambda x: len(x)).max())
print(train_df.negative.apply(lambda x: len(x)).mean(), train_df.negative.apply(lambda x: len(x)).max())

135.0042570803524 6000
86.50801158871873 6000


In [8]:
train_df, val = train_test_split(train_df, test_size=0.1, random_state=42)

## Bert

In [9]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [10]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased',do_lower_case=True)
# Create a funcition to tokenize a set of text

def tokenize_text(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # create empty lists to store outputs
    input_ids = []
    attention_masks = []
    
    #for every sentence...
    
    for sent in tqdm(data):
        # 'encode_plus will':
        # (1) Tokenize the sentence
        # (2) Add the `[CLS]` and `[SEP]` token to the start and end
        # (3) Truncate/Pad sentence to max length
        # (4) Map tokens to their IDs
        # (5) Create attention mask
        # (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            sent,   #preprocess sentence
            add_special_tokens = True,         #Add `[CLS]` and `[SEP]`
            max_length= 512  ,             #Max length to truncate/pad
            pad_to_max_length = True,          #pad sentence to max length 
            return_attention_mask= True        #Return attention mask 
        )
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
        
    #convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    
    return input_ids,attention_masks

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

In [11]:
input_ids_pos, attention_masks_pos = tokenize_text(train_df.positive.values)
input_ids_neg, attention_masks_neg = tokenize_text(train_df.negative.values)
train_labels = train_df.Target_vector.tolist()

val_input_ids_pos, val_attention_masks_pos = tokenize_text(val.positive.values)
val_input_ids_neg, val_attention_masks_neg = tokenize_text(val.negative.values)
val_labels = val.Target_vector.tolist()

  0%|          | 0/45665 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 5074/5074 [00:04<00:00, 1197.18it/s]


In [12]:
train_labels = torch.Tensor(train_labels)
val_labels = torch.Tensor(val_labels)

In [13]:
batch_size = 8

train_data = TensorDataset(input_ids_pos, attention_masks_pos, input_ids_neg, attention_masks_neg, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_input_ids_pos, val_attention_masks_pos, val_input_ids_neg, val_attention_masks_neg, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [14]:

class BertClassifier(nn.Module):
    """
        Bert Model for classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param   bert: a BertModel object
        @param   classifier: a torch.nn.Module classifier
        @param   freeze_bert (bool): Set `False` to fine_tune the Bert model
        """
        super(BertClassifier,self).__init__()
        # Specify hidden size of Bert, hidden size of our classifier, and number of labels
        D_in, H,D_out = 768,30,9
        
        self.bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(2*D_in, H),
            nn.ReLU(),
            nn.Linear(H, D_out))
        
        self.sigmoid = nn.Sigmoid()
        # Freeze the Bert Model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
    
    def forward(self,input_ids_pos,attention_mask_pos, input_ids_neg,attention_mask_neg):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        outputs_pos = self.bert(input_ids=input_ids_pos,
                           attention_mask = attention_mask_pos)
        
        
        outputs_neg = self.bert(input_ids=input_ids_neg,
                           attention_mask = attention_mask_neg)
        
        
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls_pos = outputs_pos[0][:,0,:]
        last_hidden_state_cls_neg = outputs_neg[0][:,0,:]
        
        last_hidden_state_cls = torch.cat([last_hidden_state_cls_pos, last_hidden_state_cls_neg], dim=1)
        
        # Feed input to classifier to compute logits
        logit = self.classifier(last_hidden_state_cls)
        
#         logits = self.sigmoid(logit)
        
        return logit

In [15]:
def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)
    
    bert_classifier.to(device)
    
    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                     lr=5e-5, #Default learning rate
                     eps=1e-8 #Default epsilon value
                     )
    
    # Total number of training steps
    total_steps = len(train_dataloader) * epochs
    
    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps=0, # Default value
                                              num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [16]:
loss_fn = nn.BCEWithLogitsLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids_pos, b_attn_mask_pos, b_input_ids_neg, b_attn_mask_neg, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids_pos, b_attn_mask_pos, b_input_ids_neg, b_attn_mask_neg)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels.float())
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20--50000 batches
            if (step % 1000 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids_pos, b_attn_mask_pos, b_input_ids_neg, b_attn_mask_neg, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids_pos, b_attn_mask_pos, b_input_ids_neg, b_attn_mask_neg)

        # Compute loss
        loss = loss_fn(logits, b_labels.float())
        val_loss.append(loss.item())

        # Get the predictions
        #preds = torch.argmax(logits, dim=1).flatten()
        
        # Calculate the accuracy rate
        #accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        accuracy = accuracy_thresh(logits.view(-1,9),b_labels.view(-1,9))
        
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

def accuracy_thresh(y_pred, y_true, thresh:float=0.5, sigmoid:bool=True):
    "Compute accuracy when `y_pred` and `y_true` are the same size."
    if sigmoid: 
        y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.byte()).float().mean().item()
    #return np.mean(((y_pred>thresh).float()==y_true.float()).float().cpu().numpy(), axis=1).sum()

In [17]:
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
set_seed(42)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |  1000   |   0.160639   |     -      |     -     |  907.57  
   1    |  2000   |   0.128089   |     -      |     -     |  906.05  
   1    |  3000   |   0.124757   |     -      |     -     |  905.58  
   1    |  4000   |   0.118943   |     -      |     -     |  907.46  
   1    |  5000   |   0.118168   |     -      |     -     |  905.85  
   1    |  5708   |   0.111309   |     -      |     -     |  639.69  
----------------------------------------------------------------------
   1    |    -    |   0.127792   |  0.120268  |   0.96    |  5359.57 
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   2    |  1000   |   0.099761   |     -      |     -     |  906.

In [18]:
test = pd.read_csv('/kaggle/input/headhunter-employer-review-competition/HeadHunter_test.csv')

In [19]:
nan = test[test.positive.isna()].positive.values[0]

In [20]:
test = test.replace(to_replace=nan, value='')

In [21]:
input_ids_pos_test, attention_masks_pos_test = tokenize_text(test.positive.values)
input_ids_neg_test, attention_masks_neg_test = tokenize_text(test.negative.values)
# test_labels = test.Target_vector.tolist()


100%|██████████| 50651/50651 [00:49<00:00, 1023.43it/s]


In [22]:
test_data = TensorDataset(input_ids_pos_test, attention_masks_pos_test, input_ids_neg_test, attention_masks_neg_test)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [23]:
checkpoint = {'model': bert_classifier,
          'state_dict': bert_classifier.state_dict(),
          'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

In [24]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False

    model.eval()
    return model

model = load_checkpoint('checkpoint.pth')

In [25]:
def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for step, batch in enumerate(test_dataloader):
        print(step, end='\r')
        # Load batch to GPU
        b_input_ids_pos, b_attn_mask_pos, b_input_ids_neg, b_attn_mask_neg = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids_pos, b_attn_mask_pos, b_input_ids_neg, b_attn_mask_neg)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    #probs = F.softmax(all_logits, dim=1).cpu().numpy()
    probs = all_logits.sigmoid().cpu().numpy()
    

    return probs

In [26]:
probs = bert_predict(bert_classifier, test_dataloader)

In [27]:
probs_list = [list(x) for x in probs]

In [28]:
test['Logits'] = probs_list

In [29]:
test['Target_vector'] = test.Logits.apply(lambda x: [1 if el > 0.5 else 0 for el in x])

In [30]:
test.Target_vector = test.Target_vector.apply(lambda x: np.array(x))

In [31]:
test['target'] = test.Target_vector.apply(lambda x: ','.join([str(el) for el in x.nonzero()[0]]))

In [32]:
test.target = test.apply(lambda row: ','.join(str(np.argmax(row.Logits))) if row.target =='' else row.target, axis=1)

In [33]:
submit_sample = pd.read_csv('/kaggle/input/headhunter-employer-review-competition/HeadHunter_sample_submit.csv')

In [34]:
test[['review_id','target']].to_csv('submit.csv', index=False)

In [35]:
submit_sample.to_csv('sub.csv', index=False)

In [36]:
submit = test[['review_id','target']]

In [37]:
test_c = test.copy()

In [38]:
test_c[['review_id','target']].to_csv('submit.csv', index=False)

In [39]:
test_c[test_c.target==8]


,review_id,city,position,positive,negative,salary_rating,team_rating,managment_rating,career_rating,workplace_rating,rest_recovery_rating,Logits,Target_vector,target


In [40]:
test_c['res_target'] = test_c.Logits.apply()

TypeError: apply() missing 1 required positional argument: 'func'

In [ ]:
z = pd.DataFrame({'a':[4.0,5.0,6.0,7.0,8.0],'b':[6.0,0,5.0,0,1.0]})

In [ ]:
z['c'] = z.apply(lambda row: 0 if row['b'] in (0,1) else row['a'] / math.log(row['b']), axis=1)

In [ ]:
import math

### LogReg

In [ ]:
features = ['salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating']

In [ ]:
y_train[0]

In [ ]:
X_train = torch.Tensor(train_df[features].values)
y_train = torch.Tensor(train_df.Target_vector.tolist())

X_val = torch.Tensor(val[features].values)
y_val = torch.Tensor(val.Target_vector.tolist())

In [ ]:
batch_size = 128

train_data_logreg = TensorDataset(X_train, y_train)
train_sampler_logreg = RandomSampler(train_data_logreg)
train_dataloader_logreg = DataLoader(train_data_logreg, sampler=train_sampler_logreg, batch_size=batch_size)

val_data_logreg = TensorDataset(X_val, y_val)
val_sampler_logreg = SequentialSampler(val_data_logreg)
val_dataloader_logreg = DataLoader(val_data_logreg, sampler=val_sampler_logreg, batch_size=batch_size)

In [ ]:

class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        D_in, H, D_out = 6,30,9
        
        
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(H, 2*H),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(2*H, D_out))
        
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, X):
        
        logit = self.classifier(X)
        
        
        return logit

In [ ]:
def initialize_model_logreg(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    
    # Instantiate Bert Classifier
    logreg = Net()
    
    logreg.to(device)
    
    # Create the optimizer
    optimizer = AdamW(logreg.parameters(),
                     lr=2e-5, #Default learning rate
                     eps=1e-8 #Default epsilon value
                     )
    
    # Total number of training steps
    total_steps = len(train_dataloader_logreg) * epochs
    
    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps=0, # Default value
                                              num_training_steps=total_steps)
    return logreg, optimizer, scheduler

In [ ]:
loss_fn_logreg = nn.BCEWithLogitsLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train_logreg(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            X_b, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(X_b)

            # Compute loss and accumulate the loss values
            loss = loss_fn_logreg(logits, b_labels.float())
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20--50000 batches
            if (step % 1000 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate_logreg(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate_logreg(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        X_b, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(X_b)

        # Compute loss
        loss = loss_fn_logreg(logits, b_labels.float())
        val_loss.append(loss.item())

        # Get the predictions
        #preds = torch.argmax(logits, dim=1).flatten()
        
        # Calculate the accuracy rate
        #accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        accuracy = accuracy_thresh(logits.view(-1,9),b_labels.view(-1,9))
        
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

def accuracy_thresh(y_pred, y_true, thresh:float=0.5, sigmoid:bool=True):
    "Compute accuracy when `y_pred` and `y_true` are the same size."
    if sigmoid: 
        y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.byte()).float().mean().item()
    #return np.mean(((y_pred>thresh).float()==y_true.float()).float().cpu().numpy(), axis=1).sum()

In [ ]:
logreg, optimizer, scheduler = initialize_model_logreg(epochs=50)
set_seed(42)
train(logreg, train_dataloader_logreg, val_dataloader_logreg, epochs=50, evaluation=True)

### Combination of predicts